<a href="https://colab.research.google.com/github/YahiaNaiem/Wuzzuf/blob/main/Wuzzuf_scrabing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import openpyxl
import csv
import requests
from bs4 import BeautifulSoup

def insert_into_excel(data_dict, excel_file):
    # Create a new Workbook object
    workbook = openpyxl.Workbook()

    # Create a new sheet
    sheet = workbook.active

    # Iterate through the dictionary and insert data into the sheet
    for row_index, (key, value) in enumerate(data_dict.items(), start=1):
        sheet.cell(row=row_index, column=1, value=key)
        sheet.cell(row=row_index, column=2, value=value)

    # Save the workbook to the specified Excel file
    workbook.save(excel_file)


In [9]:
jobs_list = ["data scientist", "web development"]

In [11]:
# #making the request to get the HTML for page of the wanted job and parsing it using BeautifulSoup
# response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start=0")
# soup = BeautifulSoup(response.text , "html.parser")  #to use in defining the needed values(Num_of_Jobs)

# #getting the number of jobs to loop on
# num_of_jobs_text = soup.find("strong").get_text()
# Num_of_Jobs = int(num_of_jobs_text.replace(',', ''))

def insert_skills(jobs_list):
    for job_search in jobs_list:
        response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start=0")
        soup = BeautifulSoup(response.text , "html.parser")  #to use in defining the needed values(Num_of_Jobs)
        #getting the number of jobs to loop on
        num_of_jobs_text = soup.find("strong").get_text()
        Num_of_Jobs = int(num_of_jobs_text.replace(',', ''))
        jops_counter = 1
        page_number = 0
        skills_count = {}
        while jops_counter <= Num_of_Jobs: #loop for the number of jobs in all pages
            response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start={page_number}")
            soup = BeautifulSoup(response.text, "html.parser")
            page_jobs = soup.find_all("div", attrs={"class": "css-1gatmva e1v1l3u10"})
        
            for job in page_jobs: #loop for jobs in each page
                jops_counter += 1
                skills_tags = job.find("div", attrs={"class": "css-y4udm8"}).find_all("div", attrs={})[1].find_all("a", attrs={"class": "css-5x9pm1"})[0:4]
        
                try:
                    skills_text = [skill.get_text().lstrip(' · ') for skill in skills_tags]
                except:
                    continue
        
        
                # Update skills_count dictionary
                for skill in skills_text:
                    skills_count[skill.lower()] = skills_count.get(skill.lower(), 0) + 1
        
            page_number += 1
            print(f"{jops_counter - 1} jobs were scraped")
            print(f"{page_number} pages were scraped")
        
        sorted_skills = dict(sorted(skills_count.items(), key = lambda item: item[1], reverse = True))
        reduced_dict = {key: value for key, value in sorted_skills.items() if value > 4}
        insert_into_excel(reduced_dict, "data.xlsx")

In [12]:
insert_skills(["data science", "web development"])

15 jobs were scraped
1 pages were scraped
30 jobs were scraped
2 pages were scraped
45 jobs were scraped
3 pages were scraped
60 jobs were scraped
4 pages were scraped
75 jobs were scraped
5 pages were scraped
90 jobs were scraped
6 pages were scraped
105 jobs were scraped
7 pages were scraped
120 jobs were scraped
8 pages were scraped
135 jobs were scraped
9 pages were scraped
150 jobs were scraped
10 pages were scraped
165 jobs were scraped
11 pages were scraped
180 jobs were scraped
12 pages were scraped
195 jobs were scraped
13 pages were scraped
210 jobs were scraped
14 pages were scraped
225 jobs were scraped
15 pages were scraped
240 jobs were scraped
16 pages were scraped
255 jobs were scraped
17 pages were scraped
270 jobs were scraped
18 pages were scraped
285 jobs were scraped
19 pages were scraped
300 jobs were scraped
20 pages were scraped
315 jobs were scraped
21 pages were scraped
330 jobs were scraped
22 pages were scraped
345 jobs were scraped
23 pages were scraped
360

In [1]:
import json
import requests
from bs4 import BeautifulSoup

def insert_skills(jobs_list):
    all_jobs_data = {}  # Initialize dictionary to store all jobs data
    for job_search in jobs_list:
        response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start=0")
        soup = BeautifulSoup(response.text, "html.parser")
        num_of_jobs_text = soup.find("strong").get_text()
        Num_of_Jobs = int(num_of_jobs_text.replace(',', ''))
        jobs_data = {}  # Initialize dictionary to store job data for current job search
        jops_counter = 1
        page_number = 0
        while jops_counter <= Num_of_Jobs: # Loop for the number of jobs in all pages
            response = requests.get(f"https://wuzzuf.net/search/jobs/?a=hpb&q={job_search}&start={page_number}")
            soup = BeautifulSoup(response.text, "html.parser")
            page_jobs = soup.find_all("div", attrs={"class": "css-1gatmva e1v1l3u10"})
        
            for job in page_jobs: # Loop for jobs in each page
                job_title = job.find("h2", attrs={"class": "css-m604qf"}).text.strip()
                skills_tags = job.find("div", attrs={"class": "css-y4udm8"}).find_all("div", attrs={})[1].find_all("a", attrs={"class": "css-5x9pm1"})[0:4]
                skills_text = [skill.get_text().lstrip(' · ') for skill in skills_tags]
                jobs_data[job_title] = skills_text
        
                jops_counter += 1
                if jops_counter > Num_of_Jobs:  # Break loop if all jobs are scraped
                    break
        
            page_number += 1
        
        all_jobs_data[job_search] = jobs_data  # Store job data for current job search in the main dictionary

    # Save all jobs data to JSON file
    with open('jobs_data.json', 'w') as json_file:
        json.dump(all_jobs_data, json_file, indent=4)

# Example usage
jobs_list = ['data scientist', 'software engineer']  # List of job searches
insert_skills(jobs_list)
